In [1]:
import os

import matplotlib.pylab as plt
from keras.models import load_model

from util_func import *

import pyUSID as usid
import h5py

%matplotlib inline

Using TensorFlow backend.


Loading a saved model

In [2]:
model = load_model(r'../Data/Trained-Keras-model.h5')

OSError: Unable to open file (unable to open file: name = '../Data/test-1.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

Viewing summary of the model we just loaded

In [ ]:
model.summary()

Loading *real* STEM images to test out model

In [ ]:
target_size = (256,256)
STEM_real = np.load(r'/..Data/STEM_real-test.npy')
STEM_real = tf_format(STEM_real, target_size)

In [ ]:
n = 4
print('Real STEM data')
plt.figure(figsize=(20, 10))

for i in range(1, n):
    ax = plt.subplot(2, n, i)
    ax.imshow(STEM_real[i-1,:,:,0], cmap = 'gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

Applying trained model to real STEM data

In [ ]:
decoded_lattice = model.predict(STEM_real)

Getting atomic coordinates

In [ ]:
lattice_coord = get_all_coordinates(decoded_lattice, target_size, method = 'CoM', threshold = 0.5, dist_edge = 3)

Plotting the results. Looks like our model did quite decent job in analyzing the noisy images!

In [ ]:
n = 4
plt.figure(figsize=(20, 10))

for i in range(1, n):
    ax = plt.subplot(2, n, i)
    ax.imshow(STEM_real[i-1,:,:,0], cmap = 'gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
       
    ax = plt.subplot(2, n, i + n)
    lattice_coord_ = lattice_coord[i-1]
    ax.scatter(lattice_coord_[:,1], lattice_coord_[:,0], c = lattice_coord_[:,2], cmap = 'jet')
    ax.imshow(STEM_real[i-1,:,:,0], cmap = 'gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.tight_layout()

Let's save these images in a folder

In [ ]:
mdirectory = 'Figures-test/'
if not os.path.exists(mdirectory):
    os.makedirs(mdirectory)

In [ ]:
fig1001 = plt.figure(1001, figsize=(5,5))
ax = fig1001.add_axes([0, 0, 1, 1])
for i in range(STEM_real.shape[0]):
    ax.cla()
    lattice_coord_ = lattice_coord[i]
    ax.scatter(lattice_coord_[:,1], lattice_coord_[:,0], c = lattice_coord_[:,2], cmap = 'jet', marker = "+", s = 64)
    ax.imshow(STEM_real[i,:,:,0], cmap = 'gray')
    ax.axis('off')
    fig1001.savefig(mdirectory + '_image_' + str(i) + '.png')

And also, save the the coordinates together with experimental images as hdf5 file

In [ ]:
mdirectory = 'h5-test/'
if not os.path.exists(mdirectory):
    os.makedirs(mdirectory)

# Save the data into a pyUSID dataset

Once the processing is done, put all the processed data into hdf5. We can utilize pyUSID for this purpose, which is advantageous because it is simple, and at the same time versatile.

In [ ]:
#Here's a better way

#Create empty datasets, then populate them all.
'''
#STEM data
    #Image 1
        - Raw_data
        - Results group (atom coordinates)
    #Image 2
        - Raw_data
        - Results group (atom coordinates) 
        ...
'''

file_path = 'test_v3.h5'
h5_file = h5py.File(file_path)

#Set up the spec and pos stuff
row_vals = np.arange(0, STEM_real.shape[1])
col_vals = np.arange(0, STEM_real.shape[2])

pos_dims = [usid.write_utils.Dimension('Cols', 'au', row_vals),
            usid.write_utils.Dimension('Rows', 'au', col_vals)]

spec_dims = usid.write_utils.Dimension('None', 'au', 1)


In [ ]:
h5_some_group = usid.hdf_utils.create_indexed_group(h5_file, 'Measurement')

for i in range(STEM_real.shape[0]):
    h5_imag = usid.hdf_utils.create_indexed_group(h5_some_group,'Image_' + str(i))

    h5_raw = usid.hdf_utils.write_main_dataset(h5_imag,  # parent HDF5 group
                                               (STEM_real.shape[1] * STEM_real.shape[2],1),  # shape of Main dataset
                                               'Raw_Data',  # Name of main dataset
                                               'STEM_Image',  # Physical quantity contained in Main dataset
                                               'au',  # Units for the physical quantity
                                               pos_dims,  # Position dimensions
                                               spec_dims,  # Spectroscopic dimensions
                                               dtype=np.float32,  # data type / precision
                                               compression='gzip',
                                               main_dset_attrs={})

    h5_raw[:] = STEM_real[i,:,:,0].reshape(STEM_real.shape[1]*STEM_real.shape[2],1)
    h5_results_group_1 = usid.hdf_utils.create_results_group(h5_raw, 'Atom coordinates')
    h5_lat = h5_results_group_1.create_dataset('Atom_Coords', data = lattice_coord[i])
    usid.hdf_utils.write_simple_attrs(h5_lat, {'Labels': ['X', 'Y', 'Atom type']})
print(h5_raw)

In [ ]:
usid.hdf_utils.print_tree(h5_file)

In [ ]:
#close the file
h5_file.close()